# Embedding d3 in Jupyter notebooks

First, we will have to have `ipywidgets` installed, so in your conda environment run

```conda install ipywidgets```

Then, we will import the necessary functionality and define a strange looking function whos usage will become necessary for rendering multiple plots.

In [12]:
import numpy as np
from string import Template, ascii_uppercase
from IPython.core.display import HTML

def keep_count(f):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return f(*args, **kwargs)
    wrapper.count = 0
    return wrapper

## Getting a pure Javascript plot to render

Before we can get our Python data into these plots, it makes the most sense to get a pure Javascript verison to work.  This requires several pieces.  First, we will simply grab an example script from the d3 Gallery and embed it as a string. Then we will write the HTML wrapper as a string, which we will embed the Javascript into via the Python `Template` class. This provides a clean separation between the formatting and plotting portions.

In [3]:
JS_CODE1 = '''
var letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G'];
var dataset = [244, 96, 21, 121, 167, 230, 77];
var svgWidth = 800;
var svgHeight = 300;

var svg = d3.select('#chart_$COUNT')
    .append("svg")
    .attr("width", svgWidth)
    .attr("height", svgHeight)
    .attr("class", "bar-chart");
       
var barPadding = 5;
var barWidth = (svgWidth / dataset.length);

var barChart = svg.selectAll("rect")
    .data(dataset)
    .enter()
    .append("rect")
    .attr("class", "bar")
    .attr("y", function(d) {
        return svgHeight - d
    })
    .attr("height", function(d) {
        return d;
    })
    .attr("width", barWidth - barPadding)
    .attr("transform", function (d, i) {
         var translate = [barWidth * i, 0];
         return "translate("+ translate +")";
    });
'''

Below is shown the template styling.  Note the `script` tag - this is where we do the actual plotting.  We make use of the `require.js` library, which is used on the backend of Jupyter itself.  Also note that the standard method of doing something like

```html
<script src="https://d3js.org/d3.v4.min"></script>
```

will not work due to the way that D3 was written. The basic story is that the global `d3` namespace doesn't get exported.

In [4]:
@keep_count
def draw_d3(js_code, style='', args={}):
    wrapper = '''
        <style>
            $STYLE
        </style>
        <div id=chart_$COUNT></div>
        <script>
            require.config({paths: {d3: "https://d3js.org/d3.v4.min"}});
            require(["d3"], function(d3){ 
                $JS_CODE
            });
        </script>
        '''
    sub_dict = {
        'JS_CODE': js_code,
        'STYLE': style,
        'COUNT': draw_d3.count
    }
    sub_dict.update(args)
    fill_wrap = Template(wrapper).substitute(sub_dict)
    js_wrap = Template(fill_wrap).substitute(sub_dict)
    return HTML(js_wrap)


In [5]:
draw_d3(JS_CODE1)

## Adding in Python data

The addition of python data is accomplished in an easy, but hacky way.  We just serialize the data and string substitute it.

In [6]:
CSS = '''
    .bar {
      fill: steelblue;
    }
    .bar:hover {
      fill: brown;
    }
    .axis--x path {
      color: white;
      display: none;
    }
'''

JS_CODE2 = '''
var letters = $LETTERS;
var dataset = $DATA;
var svgWidth = 800;
var svgHeight = 300;

var svg = d3.select('#chart_$COUNT')
    .append("svg")
    .attr("width", svgWidth)
    .attr("height", svgHeight)
    .attr("class", "bar-chart");
       
var barPadding = 5;
var barWidth = (svgWidth / dataset.length);

var barChart = svg.selectAll("rect")
    .data(dataset)
    .enter()
    .append("rect")
    .attr("class", "bar")
    .attr("y", function(d) {
        return svgHeight - d
    })
    .attr("height", function(d) {
        return d;
    })
    .attr("width", barWidth - barPadding)
    .attr("transform", function (d, i) {
         var translate = [barWidth * i, 0];
         return "translate("+ translate +")";
    });
'''


In [7]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

letters = [a for a in ascii_uppercase]
data = 300 * np.random.random(size=len(letters))
data_dict = {
    'LETTERS': json.dumps(letters, cls=NumpyEncoder),
    'DATA': json.dumps(data, cls=NumpyEncoder)
}
draw_d3(JS_CODE2, style=CSS, args=data_dict)

In [10]:
data = 150 * np.sin(np.linspace(0, 2*np.pi, len(letters))) + 150
data_dict = {
    'LETTERS': json.dumps(letters, cls=NumpyEncoder),
    'DATA': json.dumps(data, cls=NumpyEncoder)
}
draw_d3(JS_CODE2, style=CSS, args=data_dict)

## Javascript to Python communication?

Okay, so now we have a way to let Python "talk" to the Javascript - can we do the reverse?  

In [34]:
%%javascript
var kernel = IPython.notebook.kernel;
kernel.execute('success = True');

<IPython.core.display.Javascript object>

In [36]:
msg = "WE DID IT" if success else "WE DIDN'T DO IT"
HTML('<h1 style="font-size: 800%">{}</h1>'.format(msg))